In [287]:
from nba_api.stats.static import teams
import pandas as pd
import numpy as np
import sqlite3
import re
from datetime import datetime, timezone,timedelta
from dateutil import parser
from nba_api.stats.endpoints import scoreboardv2, ScoreboardV2
from nba_api.stats.endpoints import leaguegamefinder
from nba_api.live.nba.endpoints import boxscore, BoxScore
pd.set_option('display.max_columns',None)
pd.set_option('display.max_rows',None)
conn = sqlite3.connect("nba.db")
cursor = conn.cursor()

In [288]:
yesterday = datetime.today() - timedelta(days=1) 
yesterday = yesterday.strftime("%Y-%m-%d")


In [289]:
lst = []
z = leaguegamefinder.LeagueGameFinder(date_from_nullable=yesterday, date_to_nullable=yesterday)
z = z.get_data_frames()[0]


In [290]:
x = pd.DataFrame(z)

In [291]:
x["TotalFGMade"] = x.groupby('GAME_ID')['FGM'].transform('sum')
x["TotalFGAtt"] = x.groupby('GAME_ID')['FGA'].transform('sum')
x['Total3PM'] = x.groupby('GAME_ID')['FG3M'].transform('sum')
x["TotalTOV"] = x.groupby('GAME_ID')['TOV'].transform('sum')
x["TotalFTA"] = x.groupby('GAME_ID')['FTA'].transform('sum')
x['TotalOREB'] = x.groupby('GAME_ID')['OREB'].transform('sum')
x['TotalGamePoints'] = x.groupby('GAME_ID')['PTS'].transform('sum')
x['OppOREB'] = x['TotalOREB'] - x['OREB']
x['TDREB'] = x.groupby('GAME_ID')['DREB'].transform('sum')
x['TotalGameFTM'] = x.groupby('GAME_ID')['FTM'].transform('sum')
x['TotalDREB'] = x.groupby('GAME_ID')['DREB'].transform('sum')
x['TotalAST'] = x.groupby('GAME_ID')['AST'].transform('sum')
x['TotalSTL'] = x.groupby('GAME_ID')['STL'].transform('sum')
x['TotalBLK'] = x.groupby('GAME_ID')['BLK'].transform('sum')
x['TotalPF'] = x.groupby('GAME_ID')['PF'].transform('sum')
x['TotalTOV'] = x.groupby('GAME_ID')['TOV'].transform('sum')

In [292]:
def EFG(x):
    num = float(x['FGM']) + 0.5 * float(x['FG3M'])
    denom = float(x['FGA'])
    return round(num/denom, 3)
def TOVP(x):
    denom = float(x['FGA']) + 0.44 * x['FTA'] + float(x['TOV'])
    return round(float(x['TOV']) / denom, 4)
def OREBP(x):
    x['OPPDREB'] = x['TDREB'] - x['DREB']
    return float(x['OREB']) / (float(x['OREB'] + x['OPPDREB']))
def FTRATE(x):
    return float(x['FTA']) / float(x['FGA'])
def OppEFG(x):
    num = float(x["TotalFGMade"] - x["FGM"]) + 0.5 * float(x["Total3PM"] - x["FG3M"])
    denom = float(x['TotalFGAtt'] - x["FGA"])
    return round(num/denom, 3)
def OppTOVP(x):
    num = float(x['TotalTOV'] - x['TOV'])
    denom = float(x["TotalFGAtt"] - x["FGA"]) + 0.44 * float(x["TotalFTA"] - x["FTA"]) + num
    return round(num/denom, 3)
def DREBP(x):
    return float(x['DREB']) / float(x['DREB'] + x['OppOREB'])
def OppFTRATE(x):
    return float(x['TotalFTA'] - x['FTA']) / float(x['TotalFGAtt'] - x['FGA'])
def away(x):
    return x["MATCHUP"][:4]
def home(x):
    return x["MATCHUP"][-4:]
def teamPIE(x):
    teampie = (x['PTS'] + x['FGM'] + x['FTM'] - x['FGA'] - x['FTA'] + x['DREB'] + (0.5 * x['OREB']) + x['AST'] + x['STL'] + (x['BLK']*0.5) - x['PF'] - x['TOV']) 
    gamepie = (x['TotalGamePoints'] + x['TotalFGMade'] + x['TotalGameFTM'] - x['TotalFGAtt'] - x['TotalFTA'] + x['TotalDREB'] + (0.5 * x['TotalOREB']) + x['TotalAST'] + x['TotalSTL'] + (0.5 * x['TotalBLK']) - x['TotalPF'] - x['TotalTOV'])
    return round(float(teampie) / float(gamepie), 4)


In [293]:
x['EFG'] = x.apply(EFG, axis=1)
x['TOVP'] = x.apply(TOVP, axis=1)
x['OREBP'] = x.apply(OREBP, axis=1)
x['FTRATE'] = x.apply(FTRATE, axis=1)
x['OppEFG'] = x.apply(OppEFG, axis=1)
x['OppTOVP'] = x.apply(OppTOVP, axis=1)
x['DREBP'] = x.apply(DREBP, axis=1)
x['OppFTRATE'] = x.apply(OppFTRATE, axis=1)
x["AWAY"] = x.apply(away, axis=1)
x["HOME"] = x.apply(home, axis=1)
x['TEAMPIE'] = x.apply(teamPIE, axis=1)

In [294]:
x.drop(columns=['TDREB', 'OppOREB', 'TotalOREB', 'TotalFTA', 'TotalTOV','Total3PM', 'TotalFGAtt', 'TotalFGMade'],inplace=True)

In [295]:
def combine_game_rows(df):
    df = df.copy()
    df['is_home'] = df['MATCHUP'].str.contains('vs.', na=False)
    stat_columns = [col for col in df.columns if col not in 
                   ['SEASON_ID', 'TEAM_ID', 'TEAM_ABBREVIATION', 'TEAM_NAME', 
                    'GAME_ID', 'GAME_DATE', 'MATCHUP', 'WL', 'AWAY', 'HOME']]
    home_data = df[df['is_home'] == True].copy()
    away_data = df[df['is_home'] == False].copy()
    home_rename = {col: f'home_{col}' for col in stat_columns}
    home_rename.update({
        'TEAM_ID': 'home_TEAM_ID',
        'TEAM_ABBREVIATION': 'home_TEAM',
        'TEAM_NAME': 'home_TEAM_NAME',
        'WL': 'home_WL'
    })
    home_data = home_data.rename(columns=home_rename)
    away_rename = {col: f'away_{col}' for col in stat_columns}
    away_rename.update({
        'TEAM_ID': 'away_TEAM_ID',
        'TEAM_ABBREVIATION': 'away_TEAM',
        'TEAM_NAME': 'away_TEAM_NAME',
        'WL': 'away_WL'
    })
    away_data = away_data.rename(columns=away_rename)
    merge_cols = ['GAME_ID', 'GAME_DATE']
    if 'SEASON_ID' in df.columns:
        merge_cols.append('SEASON_ID')
    combined = pd.merge(
        home_data[merge_cols + [col for col in home_data.columns if col not in merge_cols]],
        away_data[merge_cols + [col for col in away_data.columns if col not in merge_cols]],
        on=merge_cols,
        how='inner'
    )
    combined['home_win'] = (combined['home_WL'] == 'W').astype(int)
    return combined

In [296]:
# x = combine_game_rows(x)

In [297]:
db = 'nba.db'
table_name = '[2025-2026 Team Data Combined]'
query = f"SELECT * FROM {table_name}"
recent = pd.read_sql_query(query, conn)

In [ ]:
recent

In [299]:
recent = pd.concat([x, recent], axis=0, ignore_index=True)

In [300]:
ewma_cols = ['PTS', 'FGA', 'FG_PCT', 'FG3M', 'FG3A', 'FG3_PCT', 'FTM', 'FTA', 'FT_PCT',
                 'OREB', 'DREB', 'AST', 'STL', 'TOV', 'PF', 'PLUS_MINUS', 'EFG', 'TOVP',
                 'OREBP', 'FTRATE', 'OppEFG', 'OppTOVP', 'DREBP', 'OppFTRATE', 'TEAMPIE']
for z in range(x.shape[0]):
    for col in ewma_cols:
        recent.iloc[[z]][f"EWMA {col}"] = (recent.groupby(['TEAM_ID', 'SEASON_ID'])[col]
            .apply(lambda x: x.shift(1).ewm(span=7, min_periods=7).mean())
            .reset_index(level=[0, 1], drop=True)
        )


/var/folders/j1/x2dgrv6501l9gqdy9k6_31wc0000gn/T/ipykernel_17456/2091501041.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  recent.iloc[[z]][f"EWMA {col}"] = (recent.groupby(['TEAM_ID', 'SEASON_ID'])[col]
/var/folders/j1/x2dgrv6501l9gqdy9k6_31wc0000gn/T/ipykernel_17456/2091501041.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  recent.iloc[[z]][f"EWMA {col}"] = (recent.groupby(['TEAM_ID', 'SEASON_ID'])[col]
/var/folders/j1/x2dgrv6501l9gqdy9k6_31wc0000gn/T/ipykernel_17456/2091501041.py:6: SettingWithC

In [302]:
db = 'nba.db'
conn = sqlite3.connect(db)
table_name = '2025-2026 Team Data Combined'
x.to_sql(name=table_name,
         con=conn,
         if_exists='append',
         index=False
         )

OperationalError: table 2025-2026 Team Data Combined has no column named TEAM_ID